In [6]:
import pandas as pd
import requests
import json
import os
from io import BytesIO
# from ISR.models import RDN;
import numpy as np
from PIL import Image
import time

from YDKReader import Reader
from IPython.display import clear_output

database_filename = "database.csv"
database_folder = "./Images Database/"

database = pd.DataFrame()

## Part 1: Update the Database

In [11]:
def update_database():
    '''
    Update cards database for newer cards.
    '''
    print("Updating Database....")
    response = requests.get("https://db.ygoprodeck.com/api/v7/cardinfo.php")
    if response.status_code == 200:
        print("Success")
        card = response.json()
        df = pd.json_normalize(card['data'], record_path=['card_images'], meta=['name'])
        df = df[1:]
        df['upscaled_image'] = " "
        df = df.set_index('id')
        df.index = df.index.astype(str)
        #old_df = pd.read_csv(database_filename, index_col="id")
        saved_images = os.listdir('./Images Database/')
        # saved_images = old_df['upscaled_image']
        for i in saved_images:
            df.at[i.split('.png')[0], 'upscaled_image'] = './Images Database/' + i
        df.to_csv('./' + database_filename)
        return df


## Part 2: Import the Database

In [7]:
def import_database():
    '''
    Import database. A dataframe object is then returned
    '''
    database = pd.read_csv(database_filename, index_col="id")
    database.index = database.index.astype(str)
    #database.head()
    return database

In [8]:
df = import_database()
df.head()

,image_url_en,image_url_small,image_url_cropped,name,upscaled_image,image_url_ar,upscaled_image_en,upscaled_image_ar,upscaled_image_anime
id,,,,,,,,,
64163367,https://images.ygoprodeck.com/images/cards/641...,https://images.ygoprodeck.com/images/cards_sma...,https://images.ygoprodeck.com/images/cards_cro...,"""A"" Cell Incubator",,https://www.arab-duelists.com/assets/img/cards...,NaN,NaN,NaN
91231901,https://images.ygoprodeck.com/images/cards/912...,https://images.ygoprodeck.com/images/cards_sma...,https://images.ygoprodeck.com/images/cards_cro...,"""A"" Cell Recombination Device",,https://www.arab-duelists.com/assets/img/cards...,NaN,NaN,NaN
73262676,https://images.ygoprodeck.com/images/cards/732...,https://images.ygoprodeck.com/images/cards_sma...,https://images.ygoprodeck.com/images/cards_cro...,"""A"" Cell Scatter Burst",,https://www.arab-duelists.com/assets/img/cards...,NaN,NaN,NaN
98319530,https://images.ygoprodeck.com/images/cards/983...,https://images.ygoprodeck.com/images/cards_sma...,https://images.ygoprodeck.com/images/cards_cro...,"""Infernoble Arms - Almace""",,https://www.arab-duelists.com/assets/img/cards...,./Images Database/en/98319530.png,./Images Database/ar/98319530.png,NaN
37478723,https://images.ygoprodeck.com/images/cards/374...,https://images.ygoprodeck.com/images/cards_sma...,https://images.ygoprodeck.com/images/cards_cro...,"""Infernoble Arms - Durendal""",,https://www.arab-duelists.com/assets/img/cards...,./Images Database/en/37478723.png,./Images Database/ar/37478723.png,NaN


In [9]:
df.rename(columns={"image_url" : "image_url_en"}, inplace=True)
df.head()

,image_url_en,image_url_small,image_url_cropped,name,upscaled_image,image_url_ar,upscaled_image_en,upscaled_image_ar,upscaled_image_anime
id,,,,,,,,,
64163367,https://images.ygoprodeck.com/images/cards/641...,https://images.ygoprodeck.com/images/cards_sma...,https://images.ygoprodeck.com/images/cards_cro...,"""A"" Cell Incubator",,https://www.arab-duelists.com/assets/img/cards...,NaN,NaN,NaN
91231901,https://images.ygoprodeck.com/images/cards/912...,https://images.ygoprodeck.com/images/cards_sma...,https://images.ygoprodeck.com/images/cards_cro...,"""A"" Cell Recombination Device",,https://www.arab-duelists.com/assets/img/cards...,NaN,NaN,NaN
73262676,https://images.ygoprodeck.com/images/cards/732...,https://images.ygoprodeck.com/images/cards_sma...,https://images.ygoprodeck.com/images/cards_cro...,"""A"" Cell Scatter Burst",,https://www.arab-duelists.com/assets/img/cards...,NaN,NaN,NaN
98319530,https://images.ygoprodeck.com/images/cards/983...,https://images.ygoprodeck.com/images/cards_sma...,https://images.ygoprodeck.com/images/cards_cro...,"""Infernoble Arms - Almace""",,https://www.arab-duelists.com/assets/img/cards...,./Images Database/en/98319530.png,./Images Database/ar/98319530.png,NaN
37478723,https://images.ygoprodeck.com/images/cards/374...,https://images.ygoprodeck.com/images/cards_sma...,https://images.ygoprodeck.com/images/cards_cro...,"""Infernoble Arms - Durendal""",,https://www.arab-duelists.com/assets/img/cards...,./Images Database/en/37478723.png,./Images Database/ar/37478723.png,NaN


In [17]:
ar_urls = ["https://www.arab-duelists.com/assets/img/cards/{}.jpg".format(i) for i in df.index ]
df['image_url_ar'] = ar_urls

In [19]:
df.to_csv('database.csv')

#### Enable Qt File explorer to fetch deck file

In [13]:
%gui qt

from PyQt5.QtWidgets import QFileDialog

def gui_fname(dir=None):
    '''
    Select a file via a dialog and return the file name.
    '''
    if dir is None: dir ='./'
    fname = QFileDialog.getOpenFileName(None, "Select data file...", 
                dir, filter="All files (*);; SM Files (*.sm)")
    return fname[0]

## Part 3: Image setup fuctions

In [14]:
def get_image(card_image:str):
    '''
    Create a GET request to fetch the card image. An image is then returned in bytes format.
    '''
    response_success = False
    while response_success == False:
        try:
            image = requests.get(card_image, timeout=10)
            response_success = image.status_code == 200
        except requests.exceptions.ConnectionError:
            print('connection error!\ntrying in 30 seconds..')
            time.sleep(30)            
    if response_success == True: print("image retrieved!")
    image = Image.open(BytesIO(image.content))
    return image

In [15]:
def upscale_image_local(id:str):
    '''
    Upscale a given image url using RDN Super Resolution neural network. The image file path is then returned.
    '''    
    card_image = database.at[id, 'image_url']
    image = get_image(card_image)
    lr_img = np.array(image)
    rdn = RDN(weights='noise-cancel')
    sr_image = rdn.predict(lr_img, by_patch_of_size=50);
    result = Image.fromarray(sr_image)
    file_path = database_folder + str(id) + ".png"
    result.save(file_path, "PNG")
    
    return file_path

In [16]:
def process_card(id:str):
    '''
    Start processing a card given its id. The image file is then returned.
    '''
    proccessed = database.at[id, 'upscaled_image'] != " "

    image_path = ""
    if proccessed == False:
        print(database.at[id, 'name'])
        print("Image not found.\nProccessing now...")
        image_path = upscale_image_local(id)
        database.at[id, 'upscaled_image'] = image_path
        database.to_csv("./" + database_filename)
    else:
        #get the image directly
        image_path = database.at[id, 'upscaled_image']
        print("Image found at " + image_path)

    return Image.open(image_path)

In [17]:
import numpy as np
import cv2
import math
def add_border(image:Image, border_size_mm=int):
    im = np.array(image)
    row, col = im.shape[:2]
    mm_to_pixel = 3.7795275591
    border_size = math.ceil(mm_to_pixel * border_size_mm)
    border = cv2.copyMakeBorder(
        im,
        top=border_size,
        bottom=border_size,
        left=border_size,
        right=border_size,
        borderType=cv2.BORDER_REPLICATE,
        value=[200, 200, 200]
    )
    return Image.fromarray(border)

## Part 4: Create .odg file and add the deck cards to it

In [18]:
from ODGEditor import ODGEditor
import math
def create_doc_file(folder_name, deck:dict, back_sleev=""):
    '''
    Create a .odg file placing the file in 'folder_name', 
    adding cards in 'deck' and placing the desired sleeve (optional)
    '''
    odgEditor = ODGEditor(create_path=folder_name, deck=deck.get_result(), back_sleeve=back_sleev, template_file='./Templates/card by card.odg')

    page_number = math.ceil(sum(deck.get_result().values())-1)
    odgEditor.add_pages(page_number)
    odgEditor.insert_cards()
    odgEditor.create_new_doc()

    odgEditor.copy_card_files()

In [4]:
def append_to_recipt(deck:Reader, cards:dict):
    print('adding deck to recipt..')
    for id, count in deck.get_result().items():
        if cards.get(id) == None:
            cards[id] = 0
        cards[id] += count
    return cards

In [20]:
def save_image(image:Image, folder_name:str, image_count:int, file_name:str):
    # path = folder_name + '/{} copies'.format(str(image_count))
    # if not os.path.exists(path):
    #     try:
    #         os.mkdir(path) 
    #     except FileExistsError: 
    #         print('Already exists!')
    for i in range(1, image_count+1):
        image.save(folder_name + '/' + "{} ({}).png".format(file_name, i), "PNG")

In [21]:
# All of the deck cooking is done here
def process_deck(deck :Reader, folder_name:str, make_border:bool=False):
    database = import_database()
    count = 1
    for i, j in deck.get_result().items():
        id = str(i)

        if id not in database.index: database = update_database()
        
        card_name = database.at[id, 'name']
        card_image = database.at[id, 'image_url']
        print({card_name : card_image})
        
        image = get_image(card_image)
        if make_border == True :image = add_border(image, border_size_mm=10)
        clear_output(wait=True)
        print('[{} card of {} distinct cards]'.format(count, len(deck.get_result())))
        count += 1
        image.save(folder_name + '/' + "{}.png".format(id), "PNG")
        #save_image(image, file_name=id, folder_name=folder_name, image_count=j)
    #create_doc_file(folder_name, deck, back_sleev=back_sleev)
    print("Deck Done!")

In [17]:
from YDKReader import Reader
def generate_deck_recipt(deck:Reader, folder_name:str, cost_multiplier:float = 1):
    cards_count = dict()
    append_to_recipt(deck, cards_count)
    total = sum(cards_count.values())
    total_cost = total * cost_multiplier
    recipt_file = open(folder_name + '/receipt.txt', 'w')
    recipt_file.writelines('=== \t\t=========\n')
    recipt_file.writelines("Qty \t\tCard Name\n")
    recipt_file.writelines('=== \t\t=========\n')
    cards_count = dict(sorted(cards_count.items(), key= lambda x:x[1], reverse=True))
    for id, count in cards_count.items():
        recipt_file.writelines('{} x \t\t{}\n'.format(str(count), df.at[id, 'name']))
    recipt_file.writelines("\n\nTotal number of cards: \t{}\nCheck : \t\t {} LE".format(total, total_cost))
    recipt_file.close()

### Part 5: Execution of code

In [9]:
# choosing deck to work on
file_name = gui_fname()
#file_name = './YDK Files/' + file_name.split('/')[-1]
folder_name = file_name.split('.ydk')[0]

deck = Reader(file_name)   # The deck data is read and a new folder for the deck is created
back_sleev = ""

# importing a deck sleeve if desired
prompt = input("Do you want a back sleev? (y/n)\n")
if prompt == 'y':
    back_sleev = gui_fname()


(python3:39292): IBUS-WARNING **: 13:36:02.468: Unable to connect to ibus: Could not connect: Connection refused


In [23]:
database = import_database()

In [ ]:
process_deck(deck, folder_name, make_border=False)

NameError: name 'process_deck' is not defined

In [ ]:
if back_sleev != "":
    back = add_border(Image.open(back_sleev), 10)
    back.save(folder_name + "/back.png", "PNG")

In [ ]:
create_doc_file(folder_name, deck, back_sleev)

In [18]:
folder_name = "E:\\YUGIOH Deck Wizard\\Orders\\Print\\Mahmoud\\Decks"
deck = Reader(folder_name + "\\" + "All.ydk")
generate_deck_recipt(deck, folder_name, cost_multiplier=1.5)

adding deck to recipt..


## Cloud API calling (not currently used due to limited quota)

#### Upscale using the replicate Super Resolution neural network

In [ ]:
def upscale_image_replicate():
    post_data = {
        "version": "9d91795e944f3a585fa83f749617fc75821bea8b323348f39cf84f8fd0cbc2f7",
        "input" : {"image" : card_image}
    }
    post_hearders = {
        "Authorization" : "Token [my token :)]",
        "Content-Type" : "application/json"
    }

    response = requests.post('https://api.replicate.com/v1/predictions', data=json.dumps(post_data), headers=post_hearders)
    if response.status_code != 201:
        print(response.status_code)
        print(response.content)
    else:
        print(response.json())
        
    #get the processed card after upscaling
    response_dict = dict(response.json())

    # wait until the process is finished
    time.sleep(20)
    output_image = requests.get(response_dict['urls']['get'], headers=post_hearders)

    print(output_image.status_code)
    print(output_image.content)

    #get the image after prcoessing it
    img_url = dict(output_image.json())['output'][0]['file']
    
    #caching the image link
    database.at[id, "upscaled_image"] = img_url
    database.to_csv('./' + database_filename)


#### Upscale using the waifu2x Super Resolution (model uploaded on DeepAI)

In [ ]:
def upscale_image_waifu2x():
   response = requests.post(
    "https://api.deepai.org/api/waifu2x",
    data={
        'image': card_image,
    },
    headers={'api-key': 'quickstart-[my token :)]'}
   )
   time.sleep(10)
   print(response.json())
   img = response.json()['output_url']
   import urllib.request
   file_path = database_folder + str(id)
   urllib.request.urlretrieve(img, file_path + ".png")
   return file_path

In [1]:
import cv2
from PIL import Image
import numpy as np
import math
def add_border(image:Image, border_size_mm=int):
    im = np.array(image)
    mm_to_pixel = 3.7795275591
    border_size = math.ceil(mm_to_pixel * border_size_mm)
    border = cv2.copyMakeBorder(
        im,
        top=border_size,
        bottom=border_size,
        left=border_size,
        right=border_size,
        borderType=cv2.BORDER_REPLICATE,
        value=[200, 200, 200]
    )
    return Image.fromarray(border)

In [3]:
image = Image.open("E:\\YUGIOH Deck Wizard\\yugioh-deck-wizard\\Templates\\frame.png")
image = add_border(image, border_size_mm=14)
image.save("E:\\YUGIOH Deck Wizard\\yugioh-deck-wizard\\Templates\\frame-bulk.png", "PNG")